<a href="https://colab.research.google.com/github/Neha-Syam/crops/blob/main/crop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_and_preprocess_data(directory, img_size=(224, 224)):
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

    train_generator = datagen.flow_from_directory(
        directory,
        target_size=img_size,
        batch_size=32,
        class_mode='categorical',
        subset='training'
    )

    validation_generator = datagen.flow_from_directory(
        directory,
        target_size=img_size,
        batch_size=32,
        class_mode='categorical',
        subset='validation'
    )

    return train_generator, validation_generator


In [ ]:
pip install kaggle


In [ ]:
pip install pandas

In [ ]:
import json

def load_json_data(data):
    with open(data, 'r') as file:
        data = json.load(file)
    return data

# Example usage
data = load_json_data('/content/data.json')
print(data)

{'@context': {'@language': 'en', '@vocab': 'https://schema.org/', 'citeAs': 'cr:citeAs', 'column': 'cr:column', 'conformsTo': 'dct:conformsTo', 'cr': 'http://mlcommons.org/croissant/', 'data': {'@id': 'cr:data', '@type': '@json'}, 'dataBiases': 'cr:dataBiases', 'dataCollection': 'cr:dataCollection', 'dataType': {'@id': 'cr:dataType', '@type': '@vocab'}, 'dct': 'http://purl.org/dc/terms/', 'extract': 'cr:extract', 'field': 'cr:field', 'fileProperty': 'cr:fileProperty', 'fileObject': 'cr:fileObject', 'fileSet': 'cr:fileSet', 'format': 'cr:format', 'includes': 'cr:includes', 'isEnumeration': 'cr:isEnumeration', 'isLiveDataset': 'cr:isLiveDataset', 'jsonPath': 'cr:jsonPath', 'key': 'cr:key', 'md5': 'cr:md5', 'parentField': 'cr:parentField', 'path': 'cr:path', 'personalSensitiveInformation': 'cr:personalSensitiveInformation', 'recordSet': 'cr:recordSet', 'references': 'cr:references', 'regex': 'cr:regex', 'repeated': 'cr:repeated', 'replace': 'cr:replace', 'sc': 'https://schema.org/', 'sepa

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Assuming load_and_preprocess_data function is defined correctly
def build_and_train_model(train_data, validation_data):
    base_model = MobileNetV2(weights='imagenet', include_top=False)
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(train_data.num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze the layers of the base model
    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_data, validation_data=validation_data, epochs=10)

    # Save the model
    model.save('crops.h5')  # HDF5 format

    return model



# Load and preprocess the data before model training
# Ensure that 'data' is correctly replaced with your actual data path

train_gen, val_gen = load_and_preprocess_data('/content/drive/MyDrive/Data')
model = build_and_train_model(train_gen, val_gen)



Found 173 images belonging to 9 classes.
Found 40 images belonging to 9 classes.


Epoch 1/10
6/6 [==============================] - 14s 2s/step - loss: 2.2351 - accuracy: 0.3642 - val_loss: 1.9387 - val_accuracy: 0.4000
Epoch 2/10
6/6 [==============================] - 11s 2s/step - loss: 0.5753 - accuracy: 0.7919 - val_loss: 1.0383 - val_accuracy: 0.6500
Epoch 3/10
6/6 [==============================] - 10s 2s/step - loss: 0.1484 - accuracy: 0.9595 - val_loss: 1.3409 - val_accuracy: 0.5750
Epoch 4/10
6/6 [==============================] - 8s 1s/step - loss: 0.0644 - accuracy: 0.9884 - val_loss: 1.1023 - val_accuracy: 0.6250
Epoch 5/10
6/6 [==============================] - 10s 2s/step - loss: 0.0208 - accuracy: 1.0000 - val_loss: 0.9948 - val_accuracy: 0.7000
Epoch 6/10
6/6 [==============================] - 9s 1s/step - loss: 0.0141 - accuracy: 1.0000 - val_loss: 0.9757 - val_accuracy: 0.7250
Epoch 7/10
6/6 [==============================] - 9s 2s/step - loss: 0.0063 - accuracy: 1.0000 - val_loss: 0.9623 - val_accuracy: 0.7000
Epoch 8/10
6/6 [=====================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import tensorflow as tf

# Load your existing TensorFlow model
model = tf.keras.models.load_model('crops.h5')

# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model to a .tflite file
with open('cropmodel.tflite', 'wb') as f:
    f.write(tflite_model)

